In [ ]:
import numpy as np
import pandas as pd
import torch
from IPython.display import clear_output

torch.__version__


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/mirco/Documents/Tesi/code/tesi_venv/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File

In [ ]:
#Install needed libraries
# !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.6.0+cu124.html
# !pip install torch-geometric-temporal
# clear_output()

In [ ]:
#Cloning the DGDNN model
#!git clone https://github.com/pixelhero98/DGDNN.git

Cloning into 'DGDNN'...
remote: Enumerating objects: 440, done.
remote: Counting objects: 100% (199/199), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 440 (delta 94), reused 35 (delta 35), pack-reused 241 (from 1)
Receiving objects: 100% (440/440), 129.41 KiB | 2.12 MiB/s, done.
Resolving deltas: 100% (178/178), done.


# Model 1: GConvGRU

In [7]:
from torch_geometric_temporal.nn import GConvGRU

In [8]:
model = GConvGRU(100,100,1)


sum([p.numel() for p in model.parameters()])

60600

In [25]:
df_NYSE = pd.read_csv('/content/data/NYSE.csv')
df_NASDAQ = pd.read_csv('/content/data/NASDAQ.csv')

In [31]:
df_NASDAQ, df_NYSE

(      AABA
 0     AAON
 1     AAPL
 2     AAWW
 3     AAXJ
 4     AAXN
 ...    ...
 1020  ZBRA
 1021  ZEUS
 1022  ZION
 1023   ZIV
 1024  ZUMZ
 
 [1025 rows x 1 columns],
          A
 0      AAN
 1      AAP
 2      AAT
 3       AB
 4      ABB
 ...    ...
 1731  ZB-A
 1732   ZBH
 1733    ZF
 1734   ZNH
 1735   ZTR
 
 [1736 rows x 1 columns])

# Model 3

In [9]:
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal

In [10]:
DynamicGraphTemporalSignal

torch_geometric_temporal.signal.dynamic_graph_temporal_signal.DynamicGraphTemporalSignal

# Model 2: DGDNN

GitHub: https://github.com/pixelhero98/DGDNN

In [11]:
from DGDNN.Model.dgdnn import DGDNN

In [12]:
model = DGDNN(
    [20],
    [10],
    1,
    1,
    1,
    1,
    1,
    1,
    1
)

sum([p.numel() for p in model.parameters()])

13

In [13]:
!cd /content && ls -ltr && unzip data.zip

total 8
drwxr-xr-x 1 root root 4096 May 23 13:39 sample_data
drwxr-xr-x 6 root root 4096 May 29 07:53 DGDNN
unzip:  cannot find or open data.zip, data.zip.zip or data.zip.ZIP.
